In [1]:
!pip install accelerate -U

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
pat = '...'

In [3]:
!git clone https://{pat}@github.com/mahopman/CS534_GroupProject.git

Cloning into 'CS534_GroupProject'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 118 (delta 49), reused 87 (delta 33), pack-reused 0
Receiving objects: 100% (118/118), 1.81 MiB | 3.75 MiB/s, done.
Resolving deltas: 100% (49/49), done.


## Data Preprocessing
TODO we need to figure out exactly how much processing is relevent.

In [4]:
import json
import pandas as pd

data = json.load(open('/content/CS534_GroupProject/data/eval_data.json'))

data = data['prompts']
df = pd.DataFrame(data).explode('correct').reset_index()
df['text'] = df['question'] + ' ' + df['correct']
df = df[['text', 'category']]

In [5]:
## DO WE WANT TO CLEAN TEXT THIS MUCH?
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
sw = stopwords.words('english')

def clean_text(text):
    #Removing punctuations
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'')

    # Convert words to lower case and remove stop words
    text = [word.lower() for word in text.split() if word.lower() not in sw]

    return " ".join(text)

df['cleaned_text'] = df['text'].apply(clean_text)

# convert category to int
categories = df['category'].unique()
category_map = {category: i for i, category in enumerate(categories)}
df['label'] = df['category'].map(category_map)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
# split dataset
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, random_state=42)

X_train, y_train = train['cleaned_text'], train['label']
X_test, y_test = test['cleaned_text'], test['label']


## Create Dataset Class

In [7]:
import torch

class TextSubjectDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

## BERT Tokenizer

In [8]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
# not sure if we need this... will check when we have the data... normally i just set this to 512 (max length of BERT)
max_length = 0
for prompt in df['cleaned_text']:
    input_ids = tokenizer.encode(prompt, add_special_tokens=True)
    max_length = max(max_length, len(input_ids))
print('Max sentence length: ', max_length)

Max sentence length:  512


In [16]:
# create encodings
train_encodings = tokenizer(X_train.to_list(), truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(X_test.to_list(), truncation=True, padding=True, max_length=max_length)

# create dataset
train_dataset = TextSubjectDataset(train_encodings, y_train.to_list())
valid_dataset = TextSubjectDataset(valid_encodings, y_test.to_list())

## BERT Model Training

In [10]:
# send to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
from transformers import BertForSequenceClassification

target_names = y_train.unique()

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(target_names)).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import TrainingArguments

## NEED TO UPDATE THIS ARGUMENTS
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=100,               # log & save weights each logging_steps
    save_steps=100,
    eval_strategy="steps",     # evaluate each `logging_steps`
)

In [21]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

def compute_metrics(pred):
    true = pred.label_ids
    pred = pred.predictions.argmax(-1)

    precision = precision_score(true, pred, average='weighted')
    recall = recall_score(true, pred, average='weighted')
    accuracy = accuracy_score(true, pred)
    f1 = f1_score(true, pred, average='weighted')

    return {
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy,
        'f1': f1
    }

In [22]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

In [23]:
trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,Accuracy,F1
100,0.271900,0.251830,0.925668,0.926521,0.926521,0.925310
200,0.131300,0.246463,0.937993,0.938002,0.938002,0.937430
300,0.060600,0.339419,0.923088,0.919633,0.919633,0.918391
400,0.052100,0.460717,0.920788,0.915040,0.915040,0.915359
500,0.302000,0.333932,0.928059,0.921929,0.921929,0.923021
600,0.091100,0.215550,0.960879,0.960964,0.960964,0.960780
700,0.090300,0.186409,0.966874,0.966705,0.966705,0.966476


TrainOutput(global_step=762, training_loss=0.13445381386073554, metrics={'train_runtime': 364.6869, 'train_samples_per_second': 16.699, 'train_steps_per_second': 2.089, 'total_flos': 1602403874426880.0, 'train_loss': 0.13445381386073554, 'epoch': 3.0})

In [34]:
trainer.save_model('./bert-classifier')

In [26]:
# Calculate metrics
results = trainer.evaluate()

precision = results['eval_precision']
recall = results['eval_recall']
accuracy = results['eval_accuracy']
f1 = results['eval_f1']

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Precision: 0.966873570173839
Recall: 0.9667049368541906
Accuracy: 0.9667049368541906
F1 Score: 0.9664762523312781
